# SQL проект
Компанией был приобретён крупный сервис для чтения книг по подписке. Необходимо проанализировать имеющуюся базу данных с информацией о книгах, издательствах, авторах, а также пользовательскими обзорами книг. Данный анализ позволит сформулировать ценностное предложение для нового продукта. Задачи:
 - посчитать количество книг, вышедших после 1 января 2000 года
 - посчитайте количество обзоров и среднюю оценку для каждой книги
 - определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так исключим из анализа брошюры
 - определить автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками
 - посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок

### Описание данных
*Таблица **books*** — содержит данные о книгах
- **book_id** — идентификатор книги
- **author_id** — идентификатор автора
- **title** — название книги
- **num_pages** — количество страниц
- **publication_date** — дата публикации книги
- **publisher_id** — идентификатор издателя

*Таблица **authors*** — содержит данные об авторах
- **author_id** — идентификатор автора
- **author** — имя автора

*Таблица **publishers*** — содержит данные об издательствах
- **publisher_id** — идентификатор издательства
- **publisher** — название издательства

*Таблица **ratings*** — содержит данные о пользовательских оценках книг
- **rating_id** — идентификатор оценки
- **book_id** — идентификатор книги
- **username** —  имя пользователя, оставившего оценку
- **rating** — оценка книги

*Таблица **reviews*** — содержит данные о пользовательских обзорах на книги
- **review_id** — идентификатор обзора
- **book_id** — идентификатор книги
- **username** —  имя пользователя, написавшего обзор
- **text** — текст обзора

### Схема базы данных
![shema_sql](shema_sql.png)

## Выводы
`После 1 января 2000 года` вышло **819 книг**.\
\
`Обзоры есть почти у всех книг, кроме 6`. `Наибольшее количество обзоров` - **7** у `Twilight (Twilight #1)`.\
`Наивысшая средняя оценка` - **5** - таких книг много, `наименьшая средняя оценка` - **1.5** у `Harvesting the Heart`.\
\
`Наибольшее число книг толще 50 страниц` выпустило издательство **Penguin Books** - `42 книги`.\
\
`Автор с самой высокой средней оценкой книг при 50 и более оценок` это **J.K. Rowling/Mary GrandPré** и `средняя оценка` у неё **4.28**.\
\
`Пользователи`, которые поставили `болше 50 оценок` `пишут, в среднем`, **24 обзора**.
 
## Рекомендации
- судя по результатам теста `система оценок и обзоров книг пользуется популярностью`, поэтому `можно опираться на мнение пользователей` о конкретных книгах, чтобы предлагать их читателям
- стоит подумать о `заключении партнёрства` с издательством **Penguin Books**, так как это `самое продуктивное издательство`, однако сначала стоит `изучить книги этого издательства и их рейтинги среди читателей`
- если планируется введение `промо подписок`, то туда `можно включать книги авторов с самыми высокими средними оценками книг`, при условии, что этих оценок много. Например как у **J.K. Rowling/Mary GrandPré** - `средняя оценка` **4.28** `при количестве оценок более 50`

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {
    'user': 'praktikum_student', # имя пользователя
    'pwd': 'Sdf4$2;d-d30pp', # пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432, # порт подключения
    'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = ('postgresql://{}:{}@{}:{}/{}'
                     .format(
                         db_config['user'],
                         db_config['pwd'],
                         db_config['host'],
                         db_config['port'],
                         db_config['db']))

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [2]:
# снимаем ограничение на ширину столбцов
pd.set_option('display.max_colwidth', None)

## Обзор данных

In [3]:
for table in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    print('Таблица', table)
    query = (
    '''
    SELECT *
    FROM
    ''' 
    + table +
    '''
    '''
    )
    tbl = pd.io.sql.read_sql(query, con=engine)
    display(tbl.head())
    display(tbl.info())
    print('Количество дубликатов:', tbl.duplicated().sum())
    print('*' * 100)

Таблица books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


None

Количество дубликатов: 0
****************************************************************************************************
Таблица authors


...

Количество дубликатов: 0
****************************************************************************************************
Таблица reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


None

Количество дубликатов: 0
****************************************************************************************************


### Выводы
Все таблицы в доступе, в данных нет пропусков и дубликатов.

## Задачи

### Количество книг, вышедших после 1 января 2000 года

In [4]:
query = (
    '''
    SELECT
        COUNT(book_id)
    FROM
        books
    WHERE
        publication_date > '2000-01-01'
    ''' 
)
pd.io.sql.read_sql(query, con=engine)

,count
0,819


`После 1 января 2000 года` вышло **819 книг**.

### Количество обзоров и средняя оценка книг

In [5]:
query = (
    '''
    SELECT
        b.title,
        COUNT(DISTINCT rw.review_id) reviews,
        AVG(rt.rating) avg_rating
    FROM
        books b
    LEFT JOIN reviews rw ON b.book_id = rw.book_id
    LEFT JOIN ratings rt ON b.book_id = rt.book_id
    GROUP BY
        b.book_id
    ORDER BY
        2 DESC
    ''' 
)
pd.io.sql.read_sql(query, con=engine)

,title,reviews,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,Water for Elephants,6,3.977273
2,The Glass Castle,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harry Potter #3),6,4.414634
4,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,The Natural Way to Draw,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,Essential Tales and Poems,0,4.000000


`Обзоры есть почти у всех книг, кроме 6`. `Наибольшее количество обзоров` - **7** у `Twilight (Twilight #1)`.\
`Наивысшая средняя оценка` - **5** - таких книг много, `наименьшая средняя оценка` - **1.5** у `Harvesting the Heart`.

### Издательство с наибольшим числом выпущенных книг толще 50 страниц

In [6]:
query = (
    '''
    SELECT
        p.publisher,
        COUNT(b.book_id)
    FROM
        books b
    JOIN publishers p ON b.publisher_id = p.publisher_id
    WHERE
        b.num_pages > 50
    GROUP BY
        1
    ORDER BY
        2 DESC
    LIMIT 1
    ''' 
)
pd.io.sql.read_sql(query, con=engine)

,publisher,count
0,Penguin Books,42


`Наибольшее число книг толще 50 страниц` выпустило издательство **Penguin Books** - `42 книги`.

### Автор с самой высокой средней оценкой книг при 50 и более оценок

In [7]:
query = (
    '''
    SELECT
        a.author,
        ROUND(avg(book_rt.avg_rating), 2)
    FROM
        (SELECT
            title,
            AVG(rating) avg_rating,
            COUNT(rating)
        FROM
            books b
        JOIN ratings r ON b.book_id = r.book_id
        GROUP BY
            1
        HAVING COUNT(rating) >= 50) book_rt
    JOIN books b ON b.title = book_rt.title
    JOIN authors a ON a.author_id = b.author_id
    GROUP BY
        1
    ORDER BY
        2 DESC
    LIMIT 1
    ''' 
)
pd.io.sql.read_sql(query, con=engine)

,author,round
0,J.K. Rowling/Mary GrandPré,4.28


`Автор с самой высокой средней оценкой книг при 50 и более оценок` это **J.K. Rowling/Mary GrandPré** и `средняя оценка` у неё **4.28**.

### Среднее количество обзоров пользователей, которые поставили больше 50 оценок

In [8]:
query = (
    '''
    SELECT
        ROUND(AVG(u.count)) avg_count
    FROM
        (SELECT
            r.username,
            COUNT(r.review_id)
        FROM
            (SELECT
                username
            FROM
                (SELECT
                    username,
                    COUNT(rating_id) rate_cnt
                FROM
                    ratings
            GROUP BY
                1) usernames
            WHERE usernames.rate_cnt > 50) users
        JOIN reviews r ON r.username = users.username
        GROUP BY
            1) u
    '''
)
pd.io.sql.read_sql(query, con=engine)

,avg_count
0,24.0


`Пользователи`, которые поставили `болше 50 оценок` `пишут, в среднем`, **24 обзора**.

## Выводы
`После 1 января 2000 года` вышло **819 книг**.\
\
`Обзоры есть почти у всех книг, кроме 6`. `Наибольшее количество обзоров` - **7** у `Twilight (Twilight #1)`.\
`Наивысшая средняя оценка` - **5** - таких книг много, `наименьшая средняя оценка` - **1.5** у `Harvesting the Heart`.\
\
`Наибольшее число книг толще 50 страниц` выпустило издательство **Penguin Books** - `42 книги`.\
\
`Автор с самой высокой средней оценкой книг при 50 и более оценок` это **J.K. Rowling/Mary GrandPré** и `средняя оценка` у неё **4.28**.\
\
`Пользователи`, которые поставили `болше 50 оценок` `пишут, в среднем`, **24 обзора**.
 
## Рекомендации
- судя по результатам теста `система оценок и обзоров книг пользуется популярностью`, поэтому `можно опираться на мнение пользователей` о конкретных книгах, чтобы предлагать их читателям
- стоит подумать о `заключении партнёрства` с издательством **Penguin Books**, так как это `самое продуктивное издательство`, однако сначала стоит `изучить книги этого издательства и их рейтинги среди читателей`
- если планируется введение `промо подписок`, то туда `можно включать книги авторов с самыми высокими средними оценками книг`, при условии, что этих оценок много. Например как у **J.K. Rowling/Mary GrandPré** - `средняя оценка` **4.28** `при количестве оценок более 50`